<a href="https://colab.research.google.com/github/xyliugo/HunyuanDiT/blob/main/%E2%80%9CGraph_Based_Multi_Hop_Agent_Task_ipynb%E2%80%9D%E7%9A%84%E5%89%AF%E6%9C%AC.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install -q langchain neo4j langchain_community yfiles_jupyter_graphs_for_neo4j langchain-groq  sentence-transformers

In [ ]:
from langchain.callbacks.manager import AsyncCallbackManagerForToolRun, CallbackManagerForToolRun
from langchain.agents import AgentExecutor
from langchain.agents.format_scratchpad import format_log_to_messages
from langchain.agents.output_parsers import ReActJsonSingleInputOutputParser
from langchain.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain.pydantic_v1 import BaseModel, Field
from langchain.schema import AIMessage, HumanMessage
from langchain.tools.render import render_text_description_and_args
from langchain_community.chat_models import ChatOllama
from langchain_community.embeddings import HuggingFaceEmbeddings
from langchain_groq import ChatGroq
from neo4j import GraphDatabase
from yfiles_jupyter_graphs_for_neo4j import Neo4jGraphWidget
from typing import Dict, List, Optional, Type
from langchain.pydantic_v1 import BaseModel, Field
from langchain.tools import BaseTool
from langchain_community.graphs import Neo4jGraph
from langchain_community.vectorstores.neo4j_vector import remove_lucene_chars
from typing import Tuple, List
from langchain_community.vectorstores import Neo4jVector

In [ ]:
import os
os.environ["groq_api_key"] = "Your groq api key" # 免费注册获取 https://console.groq.com/keys

## Graph-Based Multi-Hop Agent Task



多跳（Multi-Hop ）问题是一类需要多个步骤才能得出答案的复杂任务。无法通过原问题直接检索就能回答，而是需将原始问题分解为更小的步骤，每个步骤分别分析执行召唤数据，直到整体的任务完成。

在处理多跳任务时，常规的基于 Vector 直接语义相似性的方法可能显得力不从心，通过 Graph 中的节点和关系来找出线索，分步解决就成了更为理想的思路。

同时像 Neo4j 这样的图形数据库可以存储高度复杂且相互关联的结构化数据以及非结构化数据，可以通过对基于节点属性的元数据过滤来缩小检索范围，更容易理清楚问题脉络，对结果也更有可解释性。

多跳问题在整体上依赖 Agent 的 ReAct 能力来判断拆解子任务，将大模型推理脑力与采取行动能力集成在一起，理解和处理信息，评估当下情况，采取适当的行动，调用合适的工具，多轮循环直至任务完成。


---
[上一篇介绍了 Agent 和 Graph 结合使用的方式](https://www.xiaohongshu.com/explore/6665e9c000000000150135a8?app_platform=ios&app_version=8.36&share_from_user_hidden=true&type=video&author_share=1&xhsshare=WeixinSession&shareRedId=ODpFREY9Rjk2NzUyOTgwNjZFOTc6R0xP&apptime=1717979151&wechatWid=5cd9ba53a281612dd8c33e00d51b9aad&wechatOrigin=menu)， 本示例继续基于 Langchain 和 Neo4j，利用属性元数据过滤构建两个 Tool，通过 Agent 来完成 Multi-Hop 任务。

- InformationTool：获取 人或组织与之有关联的其他人或组织的信息
- InformationNewsTool：获取组织关联的新闻信息，有正面或负面的。

> 测试问题：Emil Eifrem 担任首席执行官的公司最近有什么好消息吗?

> 任务思路：先要弄清楚 Emil Eifrem 担任 CEO是哪家公司，才能知道去查具体公司的正面消息


注意：
本示例使用的Neo4j数据库 Instance 是 官方 创建的 companies demo

---

欢迎关注我的[小红书：AGI悟道](https://www.xiaohongshu.com/user/profile/64cdc8b0000000000b006bff?xhsshare=WeixinSession&appuid=64cdc8b0000000000b006bff&apptime=1715318887&wechatWid=5cd9ba53a281612dd8c33e00d51b9aad&wechatOrigin=menu)，发现更多有趣的AI Demo～

### 模型
* LLM 使用 Groq 提供的 mixtral-8x7b

In [ ]:
llm = ChatGroq(temperature=0, groq_api_key=os.environ["groq_api_key"], model_name="mixtral-8x7b-32768")

### 初始化 Neo4jGraph 数据库
companies 是 Neo4J 官方创建的围绕公司的 Demo

In [ ]:
os.environ["NEO4J_URI"] = "neo4j+s://demo.neo4jlabs.com"
os.environ["NEO4J_USERNAME"] = "companies"
os.environ["NEO4J_PASSWORD"] = "companies"
os.environ["NEO4J_DATABASE"] = "companies"

graph = Neo4jGraph(enhanced_schema=True)

### 查看数据库中 Index，主要是 FULLTEXT 和 Vector  类型
entity 这个 Index 用做全局关键词匹配上
如果使用的 OpenAI Embedding，可以在文本上 用 Vector，其他的 Embedding Dimension 不匹配 1536

In [ ]:
indexs = graph.query("SHOW INDEXES")
for index in indexs:
    if index["type"] == "FULLTEXT" or index["type"] == "VECTOR":
        name = index["name"]
        index_type = index["type"]
        properties = index["properties"]
        print(f"Index Name: {name}")
        print(f"Index Type: {index_type}")
        print(f"Index Properties: {properties}")
        print("---")

Index Name: entity
Index Type: FULLTEXT
Index Properties: ['name']
---
Index Name: fewshot
Index Type: VECTOR
Index Properties: ['embedding']
---
Index Name: news
Index Type: VECTOR
Index Properties: ['embedding']
---
Index Name: news_fulltext
Index Type: FULLTEXT
Index Properties: ['text']
---
Index Name: news_google
Index Type: VECTOR
Index Properties: ['embedding_google']
---


### 节点和关系的可视化全貌

In [ ]:
try:
  import google.colab
  from google.colab import output
  output.enable_custom_widget_manager()
except:
  pass

cypher = "MATCH (s)-[r]->(t) RETURN s,r,t LIMIT 50"

driver = GraphDatabase.driver(uri = os.environ["NEO4J_URI"],
                              auth = (os.environ["NEO4J_USERNAME"], os.environ["NEO4J_PASSWORD"]))

g = Neo4jGraphWidget(driver)

g.show_cypher(cypher)

GraphWidget(layout=Layout(height='800px', width='100%'))

### 查看 Schema，对节点，属性和关系有详细的描述

In [ ]:
graph.refresh_schema()
print(graph.schema)

Node properties:
- **Person**
  - `name`: STRING Example: "Julie Spellman Sweet"
  - `id`: STRING Example: "Eaf0bpz6NNoqLVUCqNZPAew"
  - `summary`: STRING Example: "CEO at Accenture"
- **Organization**
  - `revenue`: FLOAT Example: "1.2E8"
  - `motto`: STRING Example: ""
  - `nbrEmployees`: INTEGER Example: "375"
  - `isDissolved`: BOOLEAN 
  - `id`: STRING Example: "E0ZU8eCc5OaqS1LU9qE3n3w"
  - `isPublic`: BOOLEAN 
  - `name`: STRING Example: "New Energy Group"
  - `summary`: STRING Example: "Software company based in Rome, Metropolitan City "
- **IndustryCategory**
  - `name`: STRING Example: "Electronic Products Manufacturers"
  - `id`: STRING Example: "EUNd__O4zMNW81lAXNK2GNw"
- **City**
  - `name`: STRING Example: "Seattle"
  - `summary`: STRING Example: "City in and county seat of King County, Washington"
  - `id`: STRING Example: "EZHWv2xKgN92oYDKSjhJ2gw"
- **Country**
  - `name`: STRING Example: "United States of America"
  - `id`: STRING Example: "E01d4EK33MmCosgI2KXa4-A"
  - 

### 创建 FullText 搜索关键词


从 entity FULLTEXT Index 中检索
```
Index Name: entity
Index Type: FULLTEXT
Index Properties: ['name']
```








In [ ]:

# 全局文本搜索，从entity里找到相似的 Organization
def generate_full_text_query(input: str) -> str:
    full_text_query = ""
    #按空格拆分单词列表
    words = [el for el in remove_lucene_chars(input).split() if el]
    # “～2"为2的可编辑距离，表示可容错性
    for word in words[:-1]:
        full_text_query += f" {word}~2 AND"
    full_text_query += f" {words[-1]}~2"
    return full_text_query.strip()


# 全局文本匹配获取相似度高的组织或人
def get_candidates(input: str, node_type:str="Organization", limit: int = 5) -> List[Dict[str, str]]:
    candidate_query = f"""
                      CALL db.index.fulltext.queryNodes($index, $fulltextQuery, {{limit: $limit}})
                      YIELD node
                      WHERE labels(node)[0] = $node_type
                      RETURN distinct node.name AS candidate
                      """

    ft_query = generate_full_text_query(input)
    candidates = graph.query(
        candidate_query, {"fulltextQuery": ft_query, "index": 'entity', "limit": limit, "node_type":node_type}
    )
    # 如果候选中有跟输入完全匹配的返回唯一
    direct_match = [el["candidate"] for el in candidates if el["candidate"].lower() == input.lower()]
    if direct_match:
        return direct_match

    return [el["candidate"] for el in candidates]

### 获取人或组织及其关联的其他组织和个人的信息和关系

知识图谱有以下关系



```
(:Organization)-[:HAS_CEO]->(:Person)
(:Organization)-[:HAS_INVESTOR]->(:Organization)
(:Organization)-[:HAS_BOARD_MEMBER]->(:Person)
```



In [ ]:
##获取 Person 和 Organization 具体信息
def get_information(entity: str, relationship: str, type:str) -> str:
    # HAS_CEO  HAS_INVESTOR，HAS_BOARD_MEMBER 关系都会被整理拼接成上下文
    description_query ="""
                      MATCH (m:Organization|Person)
                      WHERE m.name = $candidate
                      MATCH (m)-[r:HAS_CEO|HAS_INVESTOR|HAS_BOARD_MEMBER]-(t)
                      WHERE type(r) = $relationship
                      WITH m, type(r) as type, t.name+" : "+t.summary as names
                      WITH m, type+": "+reduce(s="", n IN names | s + n + ", ") as types
                      WITH m, collect(types) as contexts
                      WITH m, "type:" + labels(m)[0] + "\ntitle: "+ m.name + "\nsummary: "+ m.summary
                            +"\n" +
                            reduce(s="", c in contexts | s + substring(c, 0, size(c)-2) +"\n") as context
                      RETURN context LIMIT 1
                      """
    candidates = get_candidates(entity, "Person")
    print(f"Candidates: {candidates}")
    # 全文没搜索候选项
    if not candidates:
        return "No information was found about the movie or person in the database"
    # 超过一个候选项提示用户进一步选择
    elif len(candidates) > 1:
        newline = "\n"
        return (
            "Need additional information, which of these "
            f"did you mean: {newline + newline.join(str(d) for d in candidates)}"
        )
    params={"candidate": candidates[0], "relationship": relationship}
    data = graph.query(
        description_query, params=params
    )
    print(f"Cypher: {description_query}\n")
    print(f"Parameters: {params}")
    print(f"Context: {data}")
    return data[0]["context"]

# schema information tool 输入
class InformationInput(BaseModel):
    entity: str = Field(description="name of organization or a person mentioned in the question")
    entity_type: str = Field(
        description="type of the entity",enum=["Organization", "Person"]
    )
    entity_relationship: str = Field(
        description="relationship of the entity with other", enum=["HAS_CEO", "HAS_INVESTOR", "HAS_BOARD_MEMBER"]
    )


# 获取关联组织或人信息的 Tool
class InformationTool(BaseTool):
    name = "Information"
    description = (
        "Useful for when you need to find other persion and organization associated with the specified person and organization"
    )
    args_schema: Type[BaseModel] = InformationInput

    def _run(
        self,
        entity: str,
        entity_relationship: str,
        entity_type: str,
        run_manager: Optional[CallbackManagerForToolRun] = None,
    ) -> str:
        """Use the tool."""
        return get_information(entity, entity_relationship, entity_type)

    async def _arun(
        self,
        entity: str,
        entity_relationship: str,
        entity_type: str,
        run_manager: Optional[AsyncCallbackManagerForToolRun] = None,
    ) -> str:
        """Use the tool asynchronously."""
        return get_information(entity, entity_relationship, entity_type)

### 封装获取组织关联新闻的 Tool

知识图谱有以下关系

```
(:Article)-[:HAS_CHUNK]->(:Chunk)
(:Article)-[:MENTIONS]->(:Organization)
```



In [ ]:
def get_organization_news(
    organization: Optional[str] = None,
    sentiment: Optional[str] = None,
) -> str:
    # 允许情况下并行执行
    base_query = (
        "CYPHER runtime = parallel parallelRuntimeSupport=all "
        "MATCH (c:Chunk)<-[:HAS_CHUNK]-(a:Article) WHERE "
    )
    where_queries = []
    params = {"k": 5}
    if organization:
        candidates = get_candidates(organization)
        # 超过1个让用户二次确认
        if len(candidates) > 1:
            return (
                "Ask a follow up question which of the available organizations "
                f"did the user mean. Available options: {candidates}"
            )
        where_queries.append(
            "EXISTS {(a)-[:MENTIONS]->(:Organization {name: $organization})}"
        )
        params["organization"] = candidates[0]
    # 正面和负面
    if sentiment:
        if sentiment == "positive":
            where_queries.append("a.sentiment > $sentiment")
            params["sentiment"] = 0.5
        else:
            where_queries.append("a.sentiment < $sentiment")
            params["sentiment"] = -0.5

    order_snippet = " WITH c, a ORDER BY a.date DESC LIMIT toInteger($k) "

    return_snippet = "RETURN '#title ' + a.title + '\n#date ' + toString(a.date) + '\n#text ' + c.text AS output"

    complete_query = (
        base_query + " AND ".join(where_queries) + order_snippet + return_snippet
    )
    data = graph.query(complete_query, params)

    print(f"Cypher: {complete_query}\n")
    print(f"Parameters: {params}")
    context = "###Article: ".join([el["output"] for el in data])
    return context

# Schema News Tool 输入
class NewsInput(BaseModel):
    organization: Optional[str] = Field(
        description="Organization that the user wants to find information about"
    )
    sentiment: Optional[str] = Field(
        description="Sentiment of articles", enum=["positive", "negative"]
    )

# 获取组织新闻 Tool
class NewsTool(BaseTool):
    name = "NewsInformation"
    description = (
        "useful for when you need to find relevant information in the news"
    )
    args_schema: Type[BaseModel] = NewsInput

    def _run(
        self,
        organization: Optional[str] = None,
        sentiment: Optional[str] = None,
        run_manager: Optional[CallbackManagerForToolRun] = None,
    ) -> str:
        """Use the tool."""
        return get_organization_news(organization, sentiment)

    async def _arun(
        self,
        organization: Optional[str] = None,
        sentiment: Optional[str] = None,
        run_manager: Optional[CallbackManagerForToolRun] = None,
    ) -> str:
        """Use the tool asynchronously."""
        return get_organization_news(organization, sentiment)


### 创建拥有以上工具能力的 ReAct Agent

- 组合工具集
- 自定义 ReAct Prompt
- 构建 Agent

In [ ]:
# 工具集
tools = [InformationTool(),NewsTool()]

# 基于 ReAct Prompt，输出截止在 \nObservation
chat_model_with_stop = llm.bind(stop=["\nObservation"])

# ReAct：Thought/Action/Observation
System_Message = f"""Answer the following questions as best you can.
You can answer directly if the user is greeting you or similar.
Otherise, you have access to the following tools:

{render_text_description_and_args(tools).replace('{', '{{').replace('}', '}}')}

The way you use the tools is by specifying a json blob.
Specifically, this json should have a `action` key (with the name of the tool to use)
and a `action_input` key (with the input to the tool going here).
The only values that should be in the "action" field are: {[t.name for t in tools]}
The $JSON_BLOB should only contain a SINGLE action,
do NOT return a list of multiple actions.
Here is an example of a valid $JSON_BLOB:
```
{{{{
    "action": $TOOL_NAME,
    "action_input": $INPUT
}}}}
```
The $JSON_BLOB must always be enclosed with triple backticks!

ALWAYS use the following format:
Question: the input question you must answer
Thought: you should always think about what to do
Action:```
$JSON_BLOB
```
Observation: the result of the action...
(this Thought/Action/Observation can repeat N times)
Thought: I now know the final answer
Final Answer: the final answer to the original input question USING Markdown format

Begin! Reminder to always use the exact characters `Final Answer` when responding.'
"""

prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            System_Message,
        ),
        MessagesPlaceholder(variable_name="chat_history"),
        ("user", "{input}"),
        MessagesPlaceholder(variable_name="agent_scratchpad"),
    ]
)

# 格式化聊天记录
def _format_chat_history(chat_history: List[Tuple[str, str]]):
    buffer = []
    for human, ai in chat_history:
        buffer.append(HumanMessage(content=human))
        buffer.append(AIMessage(content=ai))
    return buffer


# ReAct Agent
agent = (
    {
        "input": lambda x: x["input"],
        "agent_scratchpad": lambda x: format_log_to_messages(x["intermediate_steps"]),
        "chat_history": lambda x: _format_chat_history(x["chat_history"])
        if x.get("chat_history")
        else [],
    }
    | prompt
    | chat_model_with_stop
    | ReActJsonSingleInputOutputParser()
)

# schema agent 输入
class AgentInput(BaseModel):
    input: str
    chat_history: List[Tuple[str, str]] = Field(
        ..., extra={"widget": {"type": "chat", "input": "input", "output": "output"}}
    )


agent_executor = AgentExecutor(agent=agent, tools=tools, verbose=True).with_types(
    input_type=AgentInput
)

### 测试多跳问题：Is there any good news about the company where Emil Eifrem serves as CEO recently?
Emil Eifrem 担任首席执行官的公司最近有什么好消息吗?

In [ ]:
agent_executor.invoke({"input": "Is there any good news about the company where Emil Eifrem serves as CEO recently?", "chat_history": []})



> Entering new AgentExecutor chain...
Question: Is there any good news about the company where Emil Eifrem serves as CEO recently?

Thought: I need to find the company where Emil Eifrem is the CEO and then look for recent positive news about that company. I will use the 'Information' tool to find the company and the 'NewsInformation' tool to find recent positive news about the company.

Action:
```json
{
    "action": "Information",
    "action_input": {
        "entity": "Emil Eifrem",
        "entity_type": "Person",
        "entity_relationship": "HAS_CEO"
    }
}
```Candidates: ['Emil Eifrem']
Cypher: 
                      MATCH (m:Organization|Person)
                      WHERE m.name = $candidate
                      MATCH (m)-[r:HAS_CEO|HAS_INVESTOR|HAS_BOARD_MEMBER]-(t)
                      WHERE type(r) = $relationship
                      WITH m, type(r) as type, t.name+" : "+t.summary as names
                      WITH m, type+": "+reduce(s="", n IN names | s + n + "